<a href="https://colab.research.google.com/github/Prebor97/Rainfall_Prediction/blob/main/Neuralprophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving cleaned_rainfall_data.csv to cleaned_rainfall_data.csv


In [2]:
import pandas as pd
import numpy as np
import io
df = pd.read_csv(io.BytesIO(uploaded['cleaned_rainfall_data.csv']))

In [4]:
corr_data = df.corr()
corr_data.abs()['pr_wtr'].sort_values(ascending=False)

pr_wtr     1.000000
tmp_y      0.723513
slp        0.715450
uwnd_x     0.676430
rhum_y     0.661081
uwnd_y     0.606514
vwnd       0.598701
rhum       0.540226
omega      0.522588
tmp        0.447026
omega_y    0.418694
omega_x    0.276757
uwnd       0.203505
rhum_x     0.067540
vwnd_y     0.061336
vwnd_x     0.028259
Name: pr_wtr, dtype: float64

In [7]:
df.columns

Index(['time', 'omega_x', 'omega_y', 'omega', 'pr_wtr', 'rhum_x', 'rhum_y',
       'rhum', 'slp', 'tmp_y', 'tmp', 'uwnd_x', 'uwnd_y', 'uwnd', 'vwnd_x',
       'vwnd_y', 'vwnd'],
      dtype='object')

In [13]:
Main_df = df[['time', 'tmp_y', 'slp', 'uwnd_x', 'rhum_y', 'uwnd_y','vwnd','pr_wtr']]

In [14]:
Main_df = Main_df.rename(columns = { 'time':'ds','tmp_y': 'add1', 'slp' : 'add2', 'uwnd_x': 'add3', 'rhum_y' : 'add4', 'uwnd_y' : 'add5', 'uwnd' : 'add6','pr_wtr': 'y'})

In [39]:
Univariate_df = Main_df[['ds','y']]
Multivariate_df = Main_df[['ds','add1','add2','add3','add4','add5','y']]

In [40]:
TRAIN_SIZE = round(len(Main_df)*0.75)

In [41]:
mutlivariateTrain_df = Multivariate_df[:TRAIN_SIZE]
mutlivariateTest_df = Multivariate_df[TRAIN_SIZE:] 

In [42]:
test = mutlivariateTest_df.drop(columns = 'y')

In [43]:
UnivariateTrain_df = Univariate_df[:TRAIN_SIZE]
UnivariateTest_df = Univariate_df[TRAIN_SIZE:]

In [44]:
Utest = UnivariateTest_df.drop(columns = 'y')

In [45]:
mutlivariateTrain_df.reset_index(inplace = True,drop = True)
mutlivariateTest_df.reset_index(inplace = True, drop = True)
test.reset_index(inplace = True, drop = True)
UnivariateTrain_df.reset_index(inplace = True, drop = True)
UnivariateTest_df.reset_index(inplace = True, drop = True)
Utest.reset_index(inplace = True, drop = True)

In [46]:
def Evaluation_metrics(predicted, actual):
  MAE = np.mean(abs(predicted - actual))
  MSE = np.square(np.subtract(predicted, actual)).mean()
  RMSE = np.sqrt(np.mean(np.square(predicted - actual)))
  MAPE = np.mean((np.abs(np.subtract(actual, predicted)/actual)))*100
  print('The Mean Absolute Error is ', round(MAE,2))
  print('The Mean Squared Error is ', round(MSE,2))
  print('The Root Mean Squared Error is ', round(RMSE,2))
  print('The Mean Absolute Percentage Error is ', round(MAPE,2))

In [27]:
!pip install pystan~=2.14
!pip install fbprophet 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 67.3 MB 110 kB/s 
  Attempting uninstall: pystan
    Found existing installation: pystan 3.3.0
    Uninstalling pystan-3.3.0:
      Successfully uninstalled pystan-3.3.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 1.7 MB/s 
  Attempting uninstall: cmdstanpy
    Found existing installation: cmdstanpy 1.0.7
    Uninstalling cmdstanpy-1.0.7:
      Successfully uninstalled cmdstanpy-1.0.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prophet 1.1.1 requires cmdstanpy>=1.0.4, but you have cmdstanpy 0.9.5 which is incompatible.


In [47]:
from fbprophet import Prophet

#### Univariate Forecast

In [49]:
model = Prophet()
model.fit(UnivariateTrain_df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [50]:
forecast = model.predict(Utest)

In [51]:
forecast = forecast[['ds', 'yhat']]

In [52]:
final_pred = pd.concat((forecast['yhat'],UnivariateTest_df), axis = 1 )
final_pred.head()

,yhat,ds,y
0,15.047949,2001-04-02,13.589996
1,15.308986,2001-04-03,12.440002
2,15.459457,2001-04-04,7.890015
3,15.600232,2001-04-05,10.899994
4,15.715209,2001-04-06,17.869995


In [53]:
Evaluation_metrics(final_pred.yhat, final_pred.y)

The Mean Absolute Error is  5.11
The Mean Squared Error is  43.06
The Root Mean Squared Error is  6.56
The Mean Absolute Percentage Error is  38.68


#### Multivariate forecast

In [54]:
model1 = Prophet()
model1.add_regressor('add1', standardize = False)
model1.add_regressor('add2', standardize = False)
model1.add_regressor('add3', standardize = False)
model1.add_regressor('add4', standardize = False)
model1.add_regressor('add5', standardize = False)
model1.fit(mutlivariateTrain_df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [55]:
forecast1 = model1.predict(test)

In [56]:
forecast1 = forecast1[['ds', 'yhat']]

In [57]:
final_pred1 = pd.concat((forecast1['yhat'],mutlivariateTest_df), axis = 1 )
final_pred1.head()

,yhat,ds,add1,add2,add3,add4,add5,y
0,14.833298,2001-04-02,296.19000,101142.0,5.000015,34.50,3.740006,13.589996
1,12.642643,2001-04-03,296.95000,101182.0,6.570007,22.75,8.120010,12.440002
2,13.134631,2001-04-04,294.90000,101497.0,-2.789993,17.75,4.270004,7.890015
3,12.982444,2001-04-05,294.45000,101275.0,-2.539993,18.00,1.770004,10.899994
4,16.236347,2001-04-06,296.16998,101292.0,2.920013,37.00,5.470001,17.869995


In [58]:
Evaluation_metrics(final_pred1.yhat, final_pred1.y)

The Mean Absolute Error is  3.51
The Mean Squared Error is  21.3
The Root Mean Squared Error is  4.62
The Mean Absolute Percentage Error is  23.23


#### Neural prophet

In [59]:
! pip install neuralprophet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 98 kB 3.9 MB/s 
     |████████████████████████████████| 1.6 MB 37.2 MB/s 


In [61]:
from neuralprophet import NeuralProphet

In [60]:
# Univariate Data
univ_traindf = UnivariateTrain_df[['y', 'ds']]
univ_testdf = UnivariateTest_df[['y', 'ds']]
# Multivariate Data
multiv_traindf = mutlivariateTrain_df[['y','ds','add1','add2','add3','add4','add5']] 
multiv_testdf = mutlivariateTest_df[['y','ds','add1','add2','add3','add4','add5']]

#### Univariate neural prophet forecast

In [62]:
model2 = NeuralProphet()
model2.fit(univ_traindf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.995% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.995% of the data.
INFO - (NP.df_utils._infer_frequency) - Dataframe freq automatically defined as D
INFO:NP.df_utils:Dataframe freq automatically defined as D
INFO - (NP.config.init_data_params) - Setting normalization to global as only one dataframe provided for training.
INFO:NP.config:Setting normalization to global as only one dataframe provided for training.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO:NP.utils:Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 64
INFO:NP.config:Auto-set batch_size to 64
INFO - (NP.config.set_auto_batch_epoch) - Auto-set epochs to 87
INFO:NP.config:Auto-set epochs to 87


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 8.76E-02, min: 1.86E+00
INFO:NP.utils_torch:lr-range-test results: steep: 8.76E-02, min: 1.86E+00


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 7.67E-02, min: 1.63E+00
INFO:NP.utils_torch:lr-range-test results: steep: 7.67E-02, min: 1.63E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 8.76E-02
INFO:NP.forecaster:lr-range-test selected learning rate: 8.76E-02
Epoch[87/87]: 100%|██████████| 87/87 [01:10<00:00,  1.24it/s, SmoothL1Loss=0.00988, MAE=4.8, RMSE=6.07, Loss=0.00727, RegLoss=0]


,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,0.693283,47.758205,58.475367,0.533727,0.0
1,0.215128,22.782037,28.202679,0.160049,0.0
2,0.030258,8.455950,10.419819,0.021000,0.0
3,0.012009,5.266070,6.688178,0.008539,0.0
4,0.010343,4.900953,6.217255,0.007536,0.0
...,...,...,...,...,...
82,0.009959,4.823882,6.097787,0.007330,0.0
83,0.009934,4.819286,6.092063,0.007309,0.0
84,0.009908,4.808611,6.084389,0.007290,0.0
85,0.009895,4.807117,6.074490,0.007279,0.0


In [63]:
forecast2 = model2.predict(univ_testdf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.985% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.985% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.985% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.985% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [64]:
forecast2.head()

,ds,y,residual1,yhat1,trend,season_yearly,season_weekly
0,2001-04-02,13.589996,1.884147,15.474143,19.308897,-3.741823,-0.092930
1,2001-04-03,12.440002,3.299966,15.739968,19.308382,-3.597442,0.029029
2,2001-04-04,7.890015,8.013045,15.903060,19.307871,-3.453982,0.049172
3,2001-04-05,10.899994,5.201073,16.101067,19.307358,-3.311754,0.105462
4,2001-04-06,17.869995,-1.585987,16.284008,19.306845,-3.171082,0.148245


In [65]:
Evaluation_metrics(forecast2.yhat1, forecast2.y)

The Mean Absolute Error is  5.0
The Mean Squared Error is  41.19
The Root Mean Squared Error is  6.42
The Mean Absolute Percentage Error is  38.91


#### Multivariate neural prophet forecast

In [66]:
model3 = NeuralProphet()
model3.add_lagged_regressor('add1')
model3.add_lagged_regressor('add2')
model3.add_lagged_regressor('add3')
model3.add_lagged_regressor('add4')
model3.add_lagged_regressor('add5')
model3.fit(multiv_traindf)

INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO:NP.forecaster:n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO:NP.forecaster:n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO:NP.forecaster:n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regressor is automatically set to 1
INFO - (NP.forecaster.add_lagged_regressor) - n_lags = 'auto', but there is no lags for Autoregression. Number of lags for regr

  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.30E-01, min: 2.77E+00
INFO:NP.utils_torch:lr-range-test results: steep: 1.30E-01, min: 2.77E+00


  0%|          | 0/157 [00:00<?, ?it/s]

INFO - (NP.utils_torch.lr_range_test) - lr-range-test results: steep: 1.30E-01, min: 1.86E+00
INFO:NP.utils_torch:lr-range-test results: steep: 1.30E-01, min: 1.86E+00
INFO - (NP.forecaster._init_train_loader) - lr-range-test selected learning rate: 1.19E-01
INFO:NP.forecaster:lr-range-test selected learning rate: 1.19E-01
Epoch[87/87]: 100%|██████████| 87/87 [01:55<00:00,  1.33s/it, SmoothL1Loss=0.00694, MAE=3.96, RMSE=5.09, Loss=0.00511, RegLoss=0]


,SmoothL1Loss,MAE,RMSE,Loss,RegLoss
0,0.665768,46.277900,57.817615,0.492992,0.0
1,0.151222,18.418662,22.889542,0.110190,0.0
2,0.019302,6.696328,8.433905,0.013919,0.0
3,0.013110,5.503067,6.986660,0.009644,0.0
4,0.011166,5.060386,6.454499,0.008248,0.0
...,...,...,...,...,...
82,0.007035,3.989672,5.120695,0.005183,0.0
83,0.006996,3.974820,5.110341,0.005154,0.0
84,0.006965,3.966969,5.095579,0.005132,0.0
85,0.006946,3.960592,5.090966,0.005117,0.0


In [67]:
forecast3 = model3.predict(multiv_testdf)

INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.985% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.985% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils._infer_frequency) - Major frequency D corresponds to 99.985% of the data.
INFO:NP.df_utils:Major frequency D corresponds to 99.985% of the data.
INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - D
INFO:NP.df_utils:Defined frequency is equal to major frequency - D
INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column
INFO:NP.df_utils:Returning df with no ID column


In [68]:
Evaluation_metrics(forecast3.yhat1, forecast3.y)

The Mean Absolute Error is  4.06
The Mean Squared Error is  27.59
The Root Mean Squared Error is  5.25
The Mean Absolute Percentage Error is  29.64


In [69]:
metrics = {'Model' : ['Univariate fbprophet', 'Multivariate fbprophet', 'Univariate neural prophet', 'Multivariate Neural prophet'], 'MAE' : [5.11,3.51,5.0,4.06], 'MSE': [43.06,21.3,41.19,27.59], 'RMSE':[6.56,4.62,6.42,5.25],'MAPE':[38.68,23.23,38.91,29.64]}

In [71]:
Evaluation_metrics = pd.DataFrame(metrics)
display(Evaluation_metrics.head())

,Model,MAE,MSE,RMSE,MAPE
0,Univariate fbprophet,5.11,43.06,6.56,38.68
1,Multivariate fbprophet,3.51,21.30,4.62,23.23
2,Univariate neural prophet,5.00,41.19,6.42,38.91
3,Multivariate Neural prophet,4.06,27.59,5.25,29.64


In [72]:
import matplotlib.pyplot as plt

In [85]:
Evaluation_metrics.reset_index('Model', inplace = True)

In [81]:
! pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [83]:
import plotly.express as px

In [86]:
px.bar(Evaluation_metrics, x = 'Model', y = 'MAE',title = 'Mean Absolute Error Plot')

In [87]:
px.bar(Evaluation_metrics, x = 'Model', y = 'MSE',title = 'Mean Squared Error Plot')

In [88]:
px.bar(Evaluation_metrics, x = 'Model', y = 'RMSE',title = 'Root Mean Squared Error Plot')

In [90]:
px.bar(Evaluation_metrics, x = 'Model', y = 'MAPE',title = 'Mean Absolute Percentage Error Plot')

#### Of all the models, the 'MULTIVARIATE fbprophet' had the least errors. We can also infer that the addition of regressors helps increases model accuracy.
#### Generally speaking fbprophet performed better than neural prophet 